In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

# assign to first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#VRAM memory on-demand usage(prevent Keras takes all memory at the beginning)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   
sess = tf.Session(config=config)

KTF.set_session(sess)


Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
from keras import regularizers
from keras.callbacks import ModelCheckpoint

import numpy as np

In [3]:
num_classes = 10
num_predictions = 20
batch_size = 64

(x_main, y_main), (x_test, y_test) = cifar10.load_data()

print('x shape:', x_main.shape)
x_train=x_main[:40000]
y_train=y_main[:40000]
print('x_train shape:', x_train.shape)
x_val=x_main[40000:]
y_val=y_main[40000:]
print('x_val shape:', x_val.shape)




# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_val=x_val.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_val=(x_val-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)


x shape: (50000, 32, 32, 3)
x_train shape: (40000, 32, 32, 3)
x_val shape: (10000, 32, 32, 3)


In [4]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [5]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

mc = ModelCheckpoint('no_dropout_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [6]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training


opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=150,\
                    verbose=1,validation_data=(x_val,y_val),callbacks=[LearningRateScheduler(lr_schedule),mc])

#save to disk
model_json = model.to_json()
with open('no_dropout_model.json', 'w') as json_file:
    json_file.write(model_json)



Epoch 1/150
625/625 [==============================] - 21s 33ms/step - loss: 1.6440 - acc: 0.4710 - val_loss: 1.4251 - val_acc: 0.5877

Epoch 00001: val_acc improved from -inf to 0.58770, saving model to no_dropout_model.h5
Epoch 2/150
625/625 [==============================] - 19s 30ms/step - loss: 1.1308 - acc: 0.6333 - val_loss: 1.0213 - val_acc: 0.6915

Epoch 00002: val_acc improved from 0.58770 to 0.69150, saving model to no_dropout_model.h5
Epoch 3/150
625/625 [==============================] - 19s 30ms/step - loss: 0.9656 - acc: 0.6897 - val_loss: 0.8380 - val_acc: 0.7467

Epoch 00003: val_acc improved from 0.69150 to 0.74670, saving model to no_dropout_model.h5
Epoch 4/150
625/625 [==============================] - 18s 29ms/step - loss: 0.8752 - acc: 0.7260 - val_loss: 0.8736 - val_acc: 0.7425

Epoch 00004: val_acc did not improve from 0.74670
Epoch 5/150
625/625 [==============================] - 18s 29ms/step - loss: 0.8021 - acc: 0.7530 - val_loss: 0.8429 - val_acc: 0.7597



625/625 [==============================] - 19s 31ms/step - loss: 0.4617 - acc: 0.8973 - val_loss: 0.6039 - val_acc: 0.8633

Epoch 00042: val_acc improved from 0.86180 to 0.86330, saving model to no_dropout_model.h5
Epoch 43/150
625/625 [==============================] - 19s 30ms/step - loss: 0.4590 - acc: 0.8988 - val_loss: 0.6313 - val_acc: 0.8548

Epoch 00043: val_acc did not improve from 0.86330
Epoch 44/150
625/625 [==============================] - 19s 30ms/step - loss: 0.4584 - acc: 0.8976 - val_loss: 0.6516 - val_acc: 0.8522

Epoch 00044: val_acc did not improve from 0.86330
Epoch 45/150
625/625 [==============================] - 19s 30ms/step - loss: 0.4581 - acc: 0.8989 - val_loss: 0.6582 - val_acc: 0.8514

Epoch 00045: val_acc did not improve from 0.86330
Epoch 46/150
625/625 [==============================] - 19s 30ms/step - loss: 0.4547 - acc: 0.8994 - val_loss: 0.7317 - val_acc: 0.8387

Epoch 00046: val_acc did not improve from 0.86330
Epoch 47/150
625/625 [===============

625/625 [==============================] - 19s 30ms/step - loss: 0.3210 - acc: 0.9412 - val_loss: 0.5845 - val_acc: 0.8789

Epoch 00084: val_acc did not improve from 0.89140
Epoch 85/150
625/625 [==============================] - 18s 30ms/step - loss: 0.3134 - acc: 0.9426 - val_loss: 0.5625 - val_acc: 0.8889

Epoch 00085: val_acc did not improve from 0.89140
Epoch 86/150
625/625 [==============================] - 19s 31ms/step - loss: 0.3126 - acc: 0.9433 - val_loss: 0.5477 - val_acc: 0.8872

Epoch 00086: val_acc did not improve from 0.89140
Epoch 87/150
625/625 [==============================] - 19s 31ms/step - loss: 0.3066 - acc: 0.9434 - val_loss: 0.5758 - val_acc: 0.8845

Epoch 00087: val_acc did not improve from 0.89140
Epoch 88/150
625/625 [==============================] - 19s 30ms/step - loss: 0.3067 - acc: 0.9436 - val_loss: 0.5993 - val_acc: 0.8799

Epoch 00088: val_acc did not improve from 0.89140
Epoch 89/150
625/625 [==============================] - 19s 31ms/step - loss: 

625/625 [==============================] - 19s 30ms/step - loss: 0.2692 - acc: 0.9502 - val_loss: 0.5352 - val_acc: 0.8883

Epoch 00128: val_acc did not improve from 0.89140
Epoch 129/150
625/625 [==============================] - 19s 30ms/step - loss: 0.2731 - acc: 0.9477 - val_loss: 0.6075 - val_acc: 0.8755

Epoch 00129: val_acc did not improve from 0.89140
Epoch 130/150
625/625 [==============================] - 19s 30ms/step - loss: 0.2658 - acc: 0.9504 - val_loss: 0.6411 - val_acc: 0.8732

Epoch 00130: val_acc did not improve from 0.89140
Epoch 131/150
625/625 [==============================] - 19s 30ms/step - loss: 0.2735 - acc: 0.9476 - val_loss: 0.5743 - val_acc: 0.8860

Epoch 00131: val_acc did not improve from 0.89140
Epoch 132/150
625/625 [==============================] - 19s 30ms/step - loss: 0.2709 - acc: 0.9493 - val_loss: 0.6305 - val_acc: 0.8731

Epoch 00132: val_acc did not improve from 0.89140
Epoch 133/150
625/625 [==============================] - 19s 30ms/step - l

In [7]:
scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))


10000/10000 [==============================] - 1s 64us/step

Test result: 87.740 loss: 0.630


In [9]:
model.save('no_dropout_final.h5')


In [10]:
del model
from keras.models import load_model
model= load_model('no_dropout_model.h5')
scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 1s 76us/step

Test result: 88.530 loss: 0.594
